In [44]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

pd.set_option('display.max_rows', 1000)

In [2]:
!ls

Comext Open Data.ipynb
Country Distance
Instructions on how to use the bulkdownload facility.pdf
Readme.md
our_data


In [3]:
%%time
main_dir = 'our_data/'

path_hs = main_dir + 'transportation_hs'
all_hs = glob.glob(path_hs + "/*.dat")

list_df_hs = []

for filename in all_hs:
    df_tmp_hs = pd.read_csv(filename, index_col=None, header=0)
    list_df_hs.append(df_tmp_hs)

df_hs = pd.concat(list_df_hs, axis=0, ignore_index=True)

path_nstr = main_dir + 'transportation_nstr'
all_nstr = glob.glob(path_nstr + "/*.dat")

list_df_nstr = []

for filename in all_nstr:
    df_tmp_nstr = pd.read_csv(filename, index_col=None, header=0)
    list_df_nstr.append(df_tmp_nstr)

df_nstr = pd.concat(list_df_nstr, axis=0, ignore_index=True)

<decorator-gen-54>:2: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 56.7 s, sys: 10.3 s, total: 1min 6s
Wall time: 1min 10s


# DataFrames

## Description of the columns of df_hs DataFrame

* **DECLARANT and DECLARANT_ISO**: The Declarant (or reporting country) is the country compiling and sending data to Eurostat.
* **PARTNER and PARTNER_ISO**: The partner country is the last known country of destination for exports, the country of origin for imports from non-EU countries and the country of consignment for imports from Member States.
* **PRODUCT_HS**: file txt CN, SITC, CPA_2002, CPA_2008, CPA_2.1, BEC or by SECTION.
* **FLOW**: 1 import, 2 export.
* **TRANSPORT_MODE**: file txt in the main folder
* **PERIOD**: This code refers to the reference period.
* **VALUE_IN_EUROS**: euros already numpy.int64.
* **QUANTITY_IN_KG**: kg already numpy.int64.

In [4]:
# open the ransport mode txt file
transp_mode = pd.read_csv('our_data/TRANSPORT_MODE.txt', sep= '\t').rename({'0':'TRANSPORT_MODE', 'Unknown':'TYPE_TRANSP_MODE'}, axis=1)

# merge the dataframes
df_hs = df_hs.merge(transp_mode, on='TRANSPORT_MODE')

# create a dataframe for the flow
flow = pd.DataFrame({'FLOW':[0,1], 'TYPE_FLOW':['import', 'export']})

# merge the dataframes
df_hs = df_hs.merge(flow, on='FLOW')

# clear the period in two columns MONTH and YEAR
df_hs['MONTH'] = df_hs['PERIOD'].apply(lambda x: str(x)[4:])
df_hs['YEAR'] = df_hs['PERIOD'].apply(lambda x: str(x)[:4])

# drop the column PERIOD
df_hs.drop('PERIOD', axis=1, inplace=True)

In [5]:
df_hs.head()

,DECLARANT,DECLARANT_ISO,PARTNER,PARTNER_ISO,PRODUCT_HS,FLOW,TRANSPORT_MODE,VALUE_IN_EUROS,QUANTITY_IN_KG,TYPE_TRANSP_MODE,TYPE_FLOW,MONTH,YEAR
0,1,FR,6,GB,010121,1,3,1014,116,Road,export,08,2020
1,1,FR,6,GB,010511,1,3,603588,1202,Road,export,08,2020
2,1,FR,6,GB,010512,1,3,643547,3786,Road,export,08,2020
3,1,FR,6,GB,010619,1,3,4496,6,Road,export,08,2020
4,1,FR,6,GB,010641,1,3,11953,1147,Road,export,08,2020


In [6]:
df_hs[df_hs['PRODUCT_HS']=='TOTAL']; # can I delete this, right?

## Description of the columns of df_hs DataFrame

In [7]:
df_nstr = df_nstr.merge(transp_mode, on='TRANSPORT_MODE')

# merge the dataframes
df_nstr = df_nstr.merge(flow, on='FLOW')

# clear the period in two columns MONTH and YEAR
df_nstr['MONTH'] = df_nstr['PERIOD'].apply(lambda x: str(x)[4:])
df_nstr['YEAR'] = df_nstr['PERIOD'].apply(lambda x: str(x)[:4])

# drop the column PERIOD
df_nstr.drop('PERIOD', axis=1, inplace=True)

In [8]:
# container
cont = pd.read_csv('our_data/CONTAINER_MODE.txt', header=None, sep='\t').rename({0:'CONTAINER_MODE', 
                                                                                 1:'TYPE_CONTAINER_MODE'}, axis=1)

df_nstr = df_nstr.merge(cont, on='CONTAINER_MODE')

In [9]:
df_nstr.head()

,DECLARANT,DECLARANT_ISO,PARTNER,PARTNER_ISO,PRODUCT_NSTR,FLOW,TRANSPORT_MODE,CONTAINER_MODE,TRANSPORT_MEANS_NAT,VALUE_IN_EUROS,QUANTITY_IN_KG,TYPE_TRANSP_MODE,TYPE_FLOW,MONTH,YEAR,TYPE_CONTAINER_MODE
0,1,FR,21,XC,939,1,1,1,1,1348,68,Sea,export,01,2019,Containerised
1,1,FR,21,XC,TOT,1,1,1,1,1348,68,Sea,export,01,2019,Containerised
2,1,FR,28,NO,819,1,1,1,3,7301,22480,Sea,export,01,2019,Containerised
3,1,FR,28,NO,819,1,1,1,28,92016,72000,Sea,export,01,2019,Containerised
4,1,FR,28,NO,910,1,1,1,958,261480,392,Sea,export,01,2019,Containerised


# Merging of the two dataframes

In [10]:
df_hs['PRODUCT_TYPE'] = 'HS'
df_nstr['PRODUCT_TYPE'] = 'NSTR'

df_hs = df_hs.rename({'PRODUCT_HS': 'PRODUCT_CODE'}, axis=1)
df_nstr = df_nstr.rename({'PRODUCT_NSTR': 'PRODUCT_CODE'}, axis=1)

df_tot = df_hs.append(df_nstr, ignore_index=True).drop(['CONTAINER_MODE', 'TRANSPORT_MEANS_NAT', 'TYPE_CONTAINER_MODE'], axis=1)

df_tot;

In [11]:
# remove the TOT rows

# too much time
index_del = []
for i in range(len(df_tot)):
    if df_tot['PRODUCT_CODE'][i] == 'TOT' or df_tot['PRODUCT_CODE'][i] == 'TOTAL':
        index_del.append(i)

df_tot = df_tot.drop(index_del, axis=0).reset_index(drop=True)

In [12]:
df_tot = df_tot.sort_values(by=['YEAR', 'MONTH']).reset_index(drop=True)

In [13]:
df_tot;

In [14]:
df_tot = df_tot.dropna(how='any')

# Graph

In [15]:
# graph construction
ser_nodes = df_tot['DECLARANT_ISO'].append(df_tot['PARTNER_ISO'])
nodes = ser_nodes.unique()

In [16]:
df_edges = df_tot[['DECLARANT_ISO', 'PARTNER_ISO']].drop_duplicates()
tuple_edges = [tuple(r) for r in df_edges.to_numpy()]

In [17]:
# init graph
G = nx.DiGraph() # no time

In [18]:
G.add_nodes_from(nodes)
G.add_edges_from(tuple_edges)

In [19]:
# plt.figure(figsize=(70,40))
# nx.draw(G);

In [20]:
# by time

def time_graph(month, year):
    G = nx.DiGraph()
    
    # nodes by time
    df_tot_time = df_tot[(df_tot['YEAR'] == year) & (df_tot['MONTH'] == month)]
    
    ser_nodes = df_tot_time['DECLARANT_ISO'].append(df_tot_time['PARTNER_ISO'])
    nodes = ser_nodes.unique()
    
    # edges by time
    df_edges = df_tot_time[['DECLARANT_ISO', 'PARTNER_ISO']].drop_duplicates()
    tuple_edges = [tuple(r) for r in df_edges.to_numpy()]
    
    G.add_nodes_from(nodes)
    G.add_edges_from(tuple_edges)
    
    return G

In [21]:
# plt.figure(figsize=(70,40))
# nx.draw(time_graph('02', '2019'));

## Distance

In [106]:
distance_df = pd.read_csv('Country Distance/distance.csv').sort_values(by='InputID').reset_index(drop=True)
distance_df.head()

,InputID,TargetID,Distance
0,ABW,ZWE,1.151116e+07
1,ABW,NIC,1.633202e+06
2,ABW,NER,8.373634e+06
3,ABW,NGA,8.486833e+06
4,ABW,NIU,1.148822e+07


In [48]:
# wrong doing this
distance_df['InputID'] = distance_df['InputID'].apply(lambda x: x.replace(' ', '')[:2])
distance_df['TargetID'] = distance_df['TargetID'].apply(lambda x: x.replace(' ', '')[:2])

In [75]:
distance_df = distance_df.rename({'InputID':'DECLARANT_ISO', 'TargetID':'PARTNER_ISO', 'Distance':'DISTANCE'}, axis=1)
distance_df['COMB_ISO'] = distance_df['DECLARANT_ISO'] + distance_df['PARTNER_ISO']
distance_df = distance_df.drop_duplicates(subset=['DECLARANT_ISO', 'PARTNER_ISO']).reset_index(drop=True)
df_tot['COMB_ISO'] = df_tot['DECLARANT_ISO'] + df_tot['PARTNER_ISO']

In [96]:
df_tot_distance = df_tot.merge(distance_df, on='COMB_ISO').drop(['DECLARANT_ISO_y', 'PARTNER_ISO_y'], axis=1)

In [97]:
df_tot[df_tot['DECLARANT_ISO'] != 'EU'].reset_index(drop=True)

,DECLARANT,DECLARANT_ISO,PARTNER,PARTNER_ISO,PRODUCT_CODE,FLOW,TRANSPORT_MODE,VALUE_IN_EUROS,QUANTITY_IN_KG,TYPE_TRANSP_MODE,TYPE_FLOW,MONTH,YEAR,PRODUCT_TYPE,COMB_ISO
0,1,FR,24,IS,210690,1,3,203,0,Road,export,01,2019,HS,FRIS
1,1,FR,24,IS,220110,1,3,6826,19200,Road,export,01,2019,HS,FRIS
2,1,FR,24,IS,392690,1,3,140,0,Road,export,01,2019,HS,FRIS
3,1,FR,24,IS,420299,1,3,129,0,Road,export,01,2019,HS,FRIS
4,1,FR,24,IS,610331,1,3,174,0,Road,export,01,2019,HS,FRIS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33943085,91,SI,96,MK,999,1,7,1668358,0,Fixed Mechanism,export,06,2021,NSTR,SIMK
33943086,91,SI,97,ME,999,1,7,7873065,0,Fixed Mechanism,export,06,2021,NSTR,SIME
33943087,91,SI,98,XS,999,1,7,16291747,0,Fixed Mechanism,export,06,2021,NSTR,SIXS
33943088,92,HR,93,BA,999,1,7,14305485,0,Fixed Mechanism,export,06,2021,NSTR,HRBA


In [98]:
series_comb = df_tot['COMB_ISO'].unique()
index_a = []
for i in range(len(series_comb)):
    if series_comb[i] not in distance_df['COMB_ISO'].values:
        index_a.append(series_comb[i])
        
len(index_a)

2710

In [99]:
index_a

['FRAD',
 'FRTR',
 'FRUA',
 'FRBY',
 'FRAM',
 'FRKZ',
 'FRTM',
 'FRBA',
 'FRXK',
 'FRXS',
 'FRTN',
 'FRTD',
 'FRSN',
 'FRLR',
 'FRBJ',
 'FRCG',
 'FRCD',
 'FRAO',
 'FRSC',
 'FRMZ',
 'FRMG',
 'FRKM',
 'FRSZ',
 'FRMX',
 'FRJM',
 'FRBB',
 'FRCW',
 'FRSX',
 'FRCL',
 'FRUY',
 'FRIQ',
 'FRIL',
 'FRAE',
 'FRPK',
 'FRMV',
 'FRBN',
 'FRCN',
 'FRKR',
 'FRPG',
 'FRSB',
 'FRPF',
 'FRQP',
 'FRQW',
 'FRQZ',
 'NLFO',
 'NLTR',
 'NLUA',
 'NLBY',
 'NLKZ',
 'NLBA',
 'NLXK',
 'NLXS',
 'NLTN',
 'NLLY',
 'NLSN',
 'NLCD',
 'NLMZ',
 'NLMG',
 'NLSZ',
 'NLMX',
 'NLBZ',
 'NLBS',
 'NLJM',
 'NLBB',
 'NLGD',
 'NLCW',
 'NLGY',
 'NLCL',
 'NLUY',
 'NLFK',
 'NLIQ',
 'NLIL',
 'NLAE',
 'NLPK',
 'NLMV',
 'NLCN',
 'NLKR',
 'NLPG',
 'NLSB',
 'NLQW',
 'NLQZ',
 'DEAD',
 'DETR',
 'DEUA',
 'DEBY',
 'DEAM',
 'DEKZ',
 'DEBA',
 'DEXK',
 'DEXS',
 'DETN',
 'DEMZ',
 'DEMG',
 'DEMX',
 'DEJM',
 'DEBB',
 'DECL',
 'DEIQ',
 'DEIL',
 'DEAE',
 'DEPK',
 'DECN',
 'DEKR',
 'DEQZ',
 'ITTR',
 'ITUA',
 'ITBY',
 'ITAM',
 'ITKZ',
 'ITBA',
 'ITXK',
 

In [100]:
distance_df[distance_df['DECLARANT_ISO'] == 'IT'].sort_values(by='PARTNER_ISO')

,DECLARANT_ISO,PARTNER_ISO,DISTANCE,COMB_ISO
14223,IT,AB,8.492503e+06,ITAB
14344,IT,AF,4.638330e+06,ITAF
14226,IT,AG,6.153858e+06,ITAG
14225,IT,AI,7.524447e+06,ITAI
14276,IT,AL,6.281990e+05,ITAL
14227,IT,AN,9.209714e+05,ITAN
14224,IT,AR,2.691071e+06,ITAR
14228,IT,AS,1.683347e+07,ITAS
14170,IT,AT,1.093670e+07,ITAT
14222,IT,AU,5.016441e+05,ITAU


In [101]:
# conclusion some distance aren't merging because in the distance_df there aren't

In [102]:
df_tot_distance

,DECLARANT,DECLARANT_ISO_x,PARTNER,PARTNER_ISO_x,PRODUCT_CODE,FLOW,TRANSPORT_MODE,VALUE_IN_EUROS,QUANTITY_IN_KG,TYPE_TRANSP_MODE,TYPE_FLOW,MONTH,YEAR,PRODUCT_TYPE,COMB_ISO,DISTANCE
0,1,FR,24,IS,210690,1,3,203,0,Road,export,01,2019,HS,FRIS,2.439785e+06
1,1,FR,24,IS,220110,1,3,6826,19200,Road,export,01,2019,HS,FRIS,2.439785e+06
2,1,FR,24,IS,392690,1,3,140,0,Road,export,01,2019,HS,FRIS,2.439785e+06
3,1,FR,24,IS,420299,1,3,129,0,Road,export,01,2019,HS,FRIS,2.439785e+06
4,1,FR,24,IS,610331,1,3,174,0,Road,export,01,2019,HS,FRIS,2.439785e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18977316,55,LT,832,UM,939,1,1,228,47,Sea,export,06,2021,NSTR,LTUM,1.095534e+07
18977317,600,CY,454,TC,150910,1,1,96,11,Sea,export,06,2021,HS,CYTC,9.619790e+05
18977318,600,CY,454,TC,182,1,1,96,11,Sea,export,06,2021,NSTR,CYTC,9.619790e+05
18977319,600,CY,328,BI,99YYY0,1,5,71,0,Post,export,06,2021,HS,CYBI,9.180731e+06


In [105]:
len(df_tot)-len(df_tot_distance)

19524093

In [109]:
distance_df['InputID'].unique()

array([' ABW', ' AFG', ' AGO', ' AIA', ' ALB', ' AND', ' ANT', ' ARE',
       ' ARG', ' ARM', ' ASM', ' ATA', ' ATF', ' ATG', ' AUS', ' AUT',
       ' AZE', ' BDI', ' BEL', ' BEN', ' BFA', ' BGD', ' BGR', ' BHR',
       ' BHS', ' BIH', ' BLR', ' BLZ', ' BMU', ' BOL', ' BRA', ' BRB',
       ' BRN', ' BTN', ' BVT', ' BWA', ' CAF', ' CAN', ' CCK', ' CHE',
       ' CHL', ' CHN', ' CIV', ' CMR', ' COD', ' COG', ' COK', ' COL',
       ' COM', ' CPV', ' CRI', ' CUB', ' CXR', ' CYM', ' CYP', ' CZE',
       ' DEU', ' DJI', ' DMA', ' DNK', ' DOM', ' DZA', ' ECU', ' EGY',
       ' ERI', ' ESH', ' ESP', ' EST', ' ETH', ' FIN', ' FJI', ' FLK',
       ' FRA', ' FRO', ' FSM', ' GAB', ' GBR', ' GEO', ' GGY', ' GHA',
       ' GIB', ' GIN', ' GLP', ' GMB', ' GNB', ' GNQ', ' GRC', ' GRD',
       ' GRL', ' GTM', ' GUF', ' GUM', ' GUY', ' HKG', ' HMD', ' HND',
       ' HRV', ' HTI', ' HUN', ' IDN', ' IMN', ' IND', ' IOT', ' IRL',
       ' IRN', ' IRQ', ' ISL', ' ISR', ' ITA', ' JAM', ' JEY', ' JOR',
      